In [1]:
#!g2.1
import math
import os
from copy import deepcopy
import torch
from ultralytics import YOLO, __version__
from ultralytics.nn.modules import Detect
from ultralytics.nn.tasks import attempt_load_one_weight
from ultralytics.engine.trainer import BaseTrainer
from ultralytics.utils import yaml_load, LOGGER, RANK
from ultralytics.utils.checks import check_yaml
from ultralytics.utils.torch_utils import initialize_weights, de_parallel
import torch_pruning as tp
from prune_trainer import DetectionTrainer_pr
from loguru import logger
from yolov8_pruning import (save_pruning_performance_graph,  replace_c2f_with_c2f_v2, save_model_v2, final_eval_v2)

In [2]:

import warnings
from ultralytics.utils.checks import check_imgsz
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

In [3]:

def train_v2(self: YOLO, pruning=False, **kwargs):
    """
    Disabled loading new model when pruning flag is set. originated from ultralytics/yolo/engine/model.py
    """

    self._check_is_pytorch_model()
    if self.session:  # Ultralytics HUB session
        if any(kwargs):
            LOGGER.warning('WARNING ⚠️ using HUB training arguments, ignoring local training arguments.')
        kwargs = self.session.train_args
    overrides = self.overrides.copy()
    overrides.update(kwargs)
    if kwargs.get('cfg'):
        LOGGER.info(f"cfg file passed. Overriding default params with {kwargs['cfg']}.")
        overrides = yaml_load(check_yaml(kwargs['cfg']))
    overrides['mode'] = 'train'
    if not overrides.get('data'):
        raise AttributeError("Dataset required but missing, i.e. pass 'data=coco128.yaml'")
    if overrides.get('resume'):
        overrides['resume'] = self.ckpt_path

    self.task = overrides.get('task') or self.task
    overrides['verbose'] = False
    #self.trainer = TASK_MAP[self.task][1](overrides=overrides, _callbacks=self.callbacks)
    self.trainer = DetectionTrainer_pr(overrides=overrides, _callbacks=self.callbacks)


    if not pruning:
        if not overrides.get('resume'):  # manually set model only if not resuming
            self.trainer.model = self.trainer.get_model(weights=self.model if self.ckpt else None, cfg=self.model.yaml)
            self.model = self.trainer.model

    else:
        # pruning mode
        self.trainer.pruning = True
        self.trainer.model = self.model

        # replace some functions to disable half precision saving
        self.trainer.save_model = save_model_v2.__get__(self.trainer)
        self.trainer.final_eval = final_eval_v2.__get__(self.trainer)

    self.trainer.hub_session = self.session  # attach optional HUB session
    #return self.trainer
    self.trainer.train()
    # Update model and cfg after training
    if RANK in (-1, 0):
        self.model, _ = attempt_load_one_weight(str(self.trainer.best))
        self.overrides = self.model.args
        self.metrics = getattr(self.trainer.validator, 'metrics', None)


In [4]:

def compare_models_mmacs(model, model_speed, input_example, warmup_steps : int = 10):
    [(model(input_example), model_speed(input_example)) for _ in range(warmup_steps)]
    
    ops_base, params_base = tp.utils.count_ops_and_params(model, input_example)
    logger.info(f"Baseline model complexity: {ops_base/1e6} MMAC, {params_base/1e6} M params")

    ops_speedup, params_speedup = tp.utils.count_ops_and_params(model_speed, input_example)
    logger.info(f"Pruned model complexity: {ops_speedup/1e6} MMAC, {params_speedup/1e6} M params")

    current_speed_up = float(ops_base) / float(ops_speedup)
    logger.info(f"speed up={current_speed_up}")

In [5]:
def prune(
    data_yml,
    max_map_drop=0.02,
    epochs=25,
    model_path="",
    save_dir="./",
    cfg_yml="./new_cfg.yaml",
    target_prune_rate=0.5,
    iterative_steps=16,
    project='./'
):
    # load trained yolov8 model
    model = YOLO(model_path)
    model.__setattr__("train_v2", train_v2.__get__(model))
    pruning_cfg = yaml_load(check_yaml(cfg_yml))
    batch_size = pruning_cfg["batch"]

    # use coco128 dataset for 10 epochs fine-tuning each pruning iteration step
    # this part is only for sample code, number of epochs should be included in config file
    pruning_cfg["data"] = data_yml
    pruning_cfg["epochs"] = epochs
    model.model.train()
    replace_c2f_with_c2f_v2(model.model)
    initialize_weights(model.model)  # set BN.eps, momentum, ReLU.inplace
    for name, param in model.model.named_parameters():
        param.requires_grad = True
    pruning_cfg["imgsz"] = check_imgsz(pruning_cfg["imgsz"]) 
    example_inputs = torch.randn(1, 3, pruning_cfg["imgsz"], pruning_cfg["imgsz"]).to(
        model.device
    )
    macs_list, nparams_list, map_list, pruned_map_list = [], [], [], []
    base_macs, base_nparams = tp.utils.count_ops_and_params(model.model, example_inputs)

    # do validation before pruning model
    pruning_cfg["project"] = project
    pruning_cfg["name"] = f"{save_dir}baseline_val"
    pruning_cfg["batch"] = 1
    validation_model = deepcopy(model)
    
    LOGGER.info(f"\n\nvalidation strats...")
    metric = validation_model.val(**pruning_cfg)
    init_map = metric.box.map50
    macs_list.append(base_macs)
    nparams_list.append(100)
    map_list.append(init_map)
    pruned_map_list.append(init_map)
    LOGGER.info(
        f"Before Pruning: MACs={base_macs / 1e9: .5f} G, #Params={base_nparams / 1e6: .5f} M, mAP={init_map: .5f}"
    )

    # prune same ratio of filter based on initial size
    pruning_ratio = 1 - math.pow((1 - target_prune_rate), 1 / iterative_steps)

    for i in range(iterative_steps):
        LOGGER.info(f"pruning step = {i} starts...")
        model.model.train()
        for name, param in model.model.named_parameters():
            param.requires_grad = True

        ignored_layers = []
        unwrapped_parameters = []
        for m in model.model.modules():
            if isinstance(m, (Detect,)):
                ignored_layers.append(m)

        example_inputs = example_inputs.to(model.device)
        pruner = tp.pruner.GroupNormPruner(
            model.model,
            example_inputs,
            importance=tp.importance.GroupNormImportance(),  # L2 norm pruning,
            iterative_steps=1,
            round_to =8,
            pruning_ratio=pruning_ratio,
            ignored_layers=ignored_layers,
            unwrapped_parameters=unwrapped_parameters,
        )

        # Test regularization
        # output = model.model(example_inputs)
        # (output[0].sum() + sum([o.sum() for o in output[1]])).backward()
        # pruner.regularize(model.model)

        pruner.step()
        # pre fine-tuning validation
        LOGGER.info(f"pre fine-tuning validation starts...")
        pruning_cfg["name"] = f"{save_dir}step_{i}_pre_val"
        pruning_cfg["batch"] = 1
        validation_model.model = deepcopy(model.model)
        metric = validation_model.val(**pruning_cfg)
        pruned_map = metric.box.map50
        pruned_macs, pruned_nparams = tp.utils.count_ops_and_params(
            pruner.model, example_inputs.to(model.device)
        )
        current_speed_up = float(macs_list[0]) / pruned_macs
        LOGGER.info(
            f"After pruning iter {i + 1}: MACs={pruned_macs / 1e9} G, #Params={pruned_nparams / 1e6} M, "
            f"mAP={pruned_map}, speed up={current_speed_up}"
        )

        # fine-tuning
        for name, param in model.model.named_parameters():
            param.requires_grad = True
        pruning_cfg["name"] = f"{save_dir}step_{i}_finetune"
        pruning_cfg["batch"] = batch_size  # restore batch size

        # return model, pruning_cfg

        model.train_v2(pruning=True, **pruning_cfg)

        # post fine-tuning validation
        pruning_cfg["name"] = f"{save_dir}step_{i}_post_val"
        pruning_cfg["batch"] = 1
        validation_model = YOLO(model.trainer.best)
        metric = validation_model.val(**pruning_cfg)
        current_map = metric.box.map50
        LOGGER.info(f"After fine tuning mAP={current_map}")

        macs_list.append(pruned_macs)
        nparams_list.append(pruned_nparams / base_nparams * 100)
        pruned_map_list.append(pruned_map)
        map_list.append(current_map)

        # remove pruner after single iteration
        del pruner

        save_pruning_performance_graph(
            nparams_list, map_list, macs_list, pruned_map_list
        )
        LOGGER.info(f"init_map={init_map}, current_map={current_map}, max_map_drop={max_map_drop}, delta={init_map - current_map }")
        if init_map - current_map > max_map_drop:
            LOGGER.info("Pruning early stop")
            break

    model.export(format="onnx")

In [6]:

warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
BASELINE_PT = './visdrone_yolo8s_640/visdrone_yolo8s_640/weights/best.pt'
prune(data_yml = "./data.yaml", \
      max_map_drop = 0.05, \
      epochs =  25, \
      model_path = BASELINE_PT  ,\
      project = './pruning_hackaton1/',
      save_dir='./visdrone_yolo8s_640_pruned_27ep/', \
      cfg_yml = './prune_custom.yaml', \
      target_prune_rate = 0.4 ,\
      iterative_steps = 10)



validation strats...
Ultralytics YOLOv8.0.216 🚀 Python-3.10.11 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 185 layers, 11129454 parameters, 16366 gradients, 28.5 GFLOPs


val: Scanning C:\Users\irady\GitHub\not_GitProjects\DeepSchool\HACK_part_1\VisDrone2019-DET-val\labels.cache... 548 images, 0 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 548/548 [00:27<00:00, 19.58it/s]


                   all        548      38759      0.505      0.393        0.4      0.239
            pedestrian        548       8844      0.542      0.396      0.428      0.196
                people        548       5125      0.569      0.276      0.329      0.124
               bicycle        548       1287      0.282       0.15      0.137     0.0591
                   car        548      14064      0.716      0.773      0.792      0.564
                   van        548       1975      0.501      0.461      0.451      0.317
                 truck        548        750      0.517      0.373      0.381      0.256
              tricycle        548       1045      0.375      0.326      0.283      0.159
       awning-tricycle        548        532      0.304      0.192      0.171      0.103
                   bus        548        251      0.709      0.533      0.582      0.414
                 motor        548       4886      0.533      0.446      0.445      0.196
Speed: 1.5ms preproce

val: Scanning C:\Users\irady\GitHub\not_GitProjects\DeepSchool\HACK_part_1\VisDrone2019-DET-val\labels.cache... 548 images, 0 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 548/548 [00:19<00:00, 28.39it/s]


                   all        548      38759     0.0214   0.000178     0.0109    0.00394
            pedestrian        548       8844          0          0          0          0
                people        548       5125          0          0          0          0
               bicycle        548       1287          0          0          0          0
                   car        548      14064      0.214    0.00178      0.109     0.0394
                   van        548       1975          0          0          0          0
                 truck        548        750          0          0          0          0
              tricycle        548       1045          0          0          0          0
       awning-tricycle        548        532          0          0          0          0
                   bus        548        251          0          0          0          0
                 motor        548       4886          0          0          0          0
Speed: 1.4ms preproce

...

                   all        548      38759      0.437      0.334       0.33      0.191

25 epochs completed in 0.762 hours.
Optimizer stripped from pruning_hackaton1\visdrone_yolo8s_640_pruned_27ep\step_2_finetune\weights\last.pt, 31.4MB
Optimizer stripped from pruning_hackaton1\visdrone_yolo8s_640_pruned_27ep\step_2_finetune\weights\best.pt, 31.4MB

Validating pruning_hackaton1\visdrone_yolo8s_640_pruned_27ep\step_2_finetune\weights\best.pt...
Ultralytics YOLOv8.0.216 🚀 Python-3.10.11 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 185 layers, 7773566 parameters, 0 gradients, 18.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:15<00:00,  1.80it/s]


                   all        548      38759      0.437      0.335       0.33      0.191
Speed: 0.5ms preprocess, 2.2ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to pruning_hackaton1\visdrone_yolo8s_640_pruned_27ep\step_2_finetune
Ultralytics YOLOv8.0.216 🚀 Python-3.10.11 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 185 layers, 7773566 parameters, 0 gradients, 18.2 GFLOPs


val: Scanning C:\Users\irady\GitHub\not_GitProjects\DeepSchool\HACK_part_1\VisDrone2019-DET-val\labels.cache... 548 images, 0 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 548/548 [00:26<00:00, 20.44it/s]


                   all        548      38759      0.459      0.328      0.333      0.192
            pedestrian        548       8844      0.482       0.32      0.346      0.148
                people        548       5125      0.512      0.232      0.281      0.104
               bicycle        548       1287      0.256     0.0979     0.0959     0.0396
                   car        548      14064      0.695      0.722      0.749      0.514
                   van        548       1975      0.419      0.398       0.37      0.255
                 truck        548        750      0.482        0.3      0.303      0.197
              tricycle        548       1045      0.392      0.225      0.219      0.115
       awning-tricycle        548        532       0.28      0.154      0.126     0.0744
                   bus        548        251       0.62      0.446      0.472       0.32
                 motor        548       4886      0.455      0.385      0.372      0.157
Speed: 1.6ms preproce

### Compare MACs and #of Parameters

In [8]:

PRUNED_PT = 'pruning_hackaton1/visdrone_yolo8s_640_pruned_27ep/step_2_finetune/weights/best.pt'
BASELINE_PT = 'visdrone_yolo8s_640/visdrone_yolo8s_640/weights/best.pt'

In [9]:
model_baseline = YOLO(BASELINE_PT).model.to("cuda:0")
model_speeduped = YOLO(PRUNED_PT).model.to("cuda:0")
example_in = torch.rand(1,3,640,640, device="cuda")

In [21]:
compare_models_mmacs(model = model_baseline, model_speed = model_speeduped,\
                     input_example = example_in )

2024-10-13 07:59:30.088 | INFO     | __main__:compare_models_mmacs:5 - Baseline model complexity: 14281.7608 MMAC, 11.13947 M params
2024-10-13 07:59:30.186 | INFO     | __main__:compare_models_mmacs:8 - Pruned model complexity: 9133.9784 MMAC, 7.78175 M params
2024-10-13 07:59:30.188 | INFO     | __main__:compare_models_mmacs:11 - speed up=1.5635860054146833


### Validation

In [22]:
model_baseline = YOLO(BASELINE_PT) #.model.to("cuda:0")
res = model_baseline.val(data ="./data.yaml", imgsz = (640, 640))

WARNING ⚠️ updating to 'imgsz=640'. 'train' and 'val' imgsz must be an integer, while 'predict' and 'export' imgsz may be a [h, w] list or an integer, i.e. 'yolo export imgsz=640,480' or 'yolo export imgsz=640'
Ultralytics YOLOv8.0.216 🚀 Python-3.10.11 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 168 layers, 11129454 parameters, 0 gradients, 28.5 GFLOPs


val: Scanning C:\Users\irady\GitHub\not_GitProjects\DeepSchool\HACK_part_1\VisDrone2019-DET-val\labels.cache... 548 images, 0 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:16<00:00,  2.09it/s]


                   all        548      38759        0.5      0.395        0.4      0.239
            pedestrian        548       8844      0.522      0.402      0.428      0.197
                people        548       5125      0.561      0.279      0.328      0.124
               bicycle        548       1287      0.272      0.148      0.131     0.0567
                   car        548      14064      0.714      0.776      0.795      0.565
                   van        548       1975      0.501      0.463      0.452      0.318
                 truck        548        750      0.507      0.379       0.39      0.257
              tricycle        548       1045      0.377      0.332      0.286      0.161
       awning-tricycle        548        532      0.312       0.19      0.172      0.102
                   bus        548        251      0.707      0.538      0.579      0.411
                 motor        548       4886      0.526      0.446      0.443      0.197
Speed: 0.6ms preproce

In [23]:
model_speeduped = YOLO(PRUNED_PT)
res = model_speeduped.val(data = "./data.yaml", imgsz = (640, 640))

WARNING ⚠️ updating to 'imgsz=640'. 'train' and 'val' imgsz must be an integer, while 'predict' and 'export' imgsz may be a [h, w] list or an integer, i.e. 'yolo export imgsz=640,480' or 'yolo export imgsz=640'
Ultralytics YOLOv8.0.216 🚀 Python-3.10.11 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 185 layers, 7773566 parameters, 0 gradients, 18.2 GFLOPs


val: Scanning C:\Users\irady\GitHub\not_GitProjects\DeepSchool\HACK_part_1\VisDrone2019-DET-val\labels.cache... 548 images, 0 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:15<00:00,  2.31it/s]


                   all        548      38759      0.439      0.335       0.33      0.191
            pedestrian        548       8844      0.452      0.336      0.345      0.147
                people        548       5125      0.499      0.242      0.279      0.104
               bicycle        548       1287      0.215      0.109     0.0933     0.0385
                   car        548      14064      0.681      0.728       0.75      0.514
                   van        548       1975       0.41      0.409      0.367      0.254
                 truck        548        750      0.456      0.307        0.3      0.196
              tricycle        548       1045      0.378      0.233      0.211      0.113
       awning-tricycle        548        532      0.263       0.15      0.122     0.0729
                   bus        548        251      0.593      0.441      0.464      0.318
                 motor        548       4886      0.441      0.396      0.371      0.155
Speed: 0.6ms preproce

## Tuning

In [6]:
def tune_after_pruning_single(model_path, cfg_yml, data_yml, epochs, resume=False  ):
    model = YOLO(model_path)
    model.__setattr__("train_v2", train_v2.__get__(model))
    pruning_cfg = yaml_load(check_yaml(cfg_yml))
    pruning_cfg["data"] = data_yml
    pruning_cfg["epochs"] = epochs
    pruning_cfg["name"] = f"tune_afterfinal_pruning"
    pruning_cfg["imgsz"] = check_imgsz(pruning_cfg["imgsz"]) 

    # pruning_cfg["lr0"] = 0.005
    # pruning_cfg["lrf"] = 0.001 
    pruning_cfg["resume"]=resume

    model.model.train()
    replace_c2f_with_c2f_v2(model.model)
    initialize_weights(model.model)  # set BN.eps, momentum, ReLU.inplace
    model.model.train()
    for name, param in model.model.named_parameters():
        param.requires_grad = True

    model.train_v2(pruning=True, **pruning_cfg)

In [7]:


warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

model_p = 'pruning_hackaton1/visdrone_yolo8s_640_pruned_27ep/step_2_finetune/weights/best.pt'
cfg_yml = './prune_custom.yaml'
data_yml = "./data.yaml"
epochs = 30
tune_after_pruning_single(model_path = model_p, cfg_yml = cfg_yml, data_yml=data_yml, epochs = epochs  )

Ultralytics YOLOv8.0.216 🚀 Python-3.10.11 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 65 weight(decay=0.0), 72 weight(decay=0.00046875), 71 bias(decay=0.0)


train: Scanning C:\Users\irady\GitHub\not_GitProjects\DeepSchool\HACK_part_1\VisDrone2019-DET-train\labels.cache... 6471 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:00<?, ?it/s]

train: WARNING ⚠️ C:\Users\irady\GitHub\not_GitProjects\DeepSchool\HACK_part_1\VisDrone2019-DET-train\images\0000137_02220_d_0000163.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Users\irady\GitHub\not_GitProjects\DeepSchool\HACK_part_1\VisDrone2019-DET-train\images\0000140_00118_d_0000002.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Users\irady\GitHub\not_GitProjects\DeepSchool\HACK_part_1\VisDrone2019-DET-train\images\9999945_00000_d_0000114.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Users\irady\GitHub\not_GitProjects\DeepSchool\HACK_part_1\VisDrone2019-DET-train\images\9999987_00000_d_0000049.jpg: 1 duplicate labels removed



val: Scanning C:\Users\irady\GitHub\not_GitProjects\DeepSchool\HACK_part_1\VisDrone2019-DET-val\labels.cache... 548 images, 0 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]


Plotting labels to tune_afterfinal_pruning\labels.jpg... 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to tune_afterfinal_pruning
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      30/30      4.85G      1.216      0.759     0.8838          7        640: 100%|██████████| 648/648 [01:35<00:00,  6.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:16<00:00,  1.69it/s]


                   all        548      38759      0.458      0.333      0.337      0.195

30 epochs completed in 0.914 hours.
Optimizer stripped from tune_afterfinal_pruning\weights\last.pt, 31.4MB
Optimizer stripped from tune_afterfinal_pruning\weights\best.pt, 31.4MB

Validating tune_afterfinal_pruning\weights\best.pt...
Ultralytics YOLOv8.0.216 🚀 Python-3.10.11 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 185 layers, 7773566 parameters, 0 gradients, 18.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:15<00:00,  1.76it/s]


                   all        548      38759      0.457      0.333      0.337      0.195
Speed: 0.5ms preprocess, 2.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to tune_afterfinal_pruning


In [8]:
model_speeduped = YOLO('tune_afterfinal_pruning/weights/best.pt')
res = model_speeduped.val(data = "./data.yaml", imgsz = (640, 640))

WARNING ⚠️ updating to 'imgsz=640'. 'train' and 'val' imgsz must be an integer, while 'predict' and 'export' imgsz may be a [h, w] list or an integer, i.e. 'yolo export imgsz=640,480' or 'yolo export imgsz=640'
Ultralytics YOLOv8.0.216 🚀 Python-3.10.11 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 185 layers, 7773566 parameters, 0 gradients, 18.2 GFLOPs


val: Scanning C:\Users\irady\GitHub\not_GitProjects\DeepSchool\HACK_part_1\VisDrone2019-DET-val\labels.cache... 548 images, 0 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:16<00:00,  2.09it/s]


                   all        548      38759      0.458      0.334      0.337      0.196
            pedestrian        548       8844      0.468      0.346      0.358      0.155
                people        548       5125       0.52      0.241      0.287      0.106
               bicycle        548       1287      0.224      0.126     0.0999     0.0399
                   car        548      14064      0.672      0.739      0.757      0.521
                   van        548       1975      0.429      0.396      0.362       0.25
                 truck        548        750        0.5      0.301      0.322      0.209
              tricycle        548       1045      0.395      0.229      0.217      0.113
       awning-tricycle        548        532      0.268      0.143      0.121     0.0716
                   bus        548        251      0.602      0.438      0.467       0.33
                 motor        548       4886      0.503      0.377      0.381      0.162
Speed: 0.7ms preproce